## Libraries

In [4]:
import sys 
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")

from custom_score.utils import get_git_root
sys.path.append(get_git_root())

In [5]:
from custom_score.score import BERTScoreStaticSampleTest, BERTScoreDynamicSampleTest, BARTScoreDynamicSampleTest
from custom_score.utils import serialized_to_model
from BARTScore.bart_score import BARTScorer
import torch
import tensorflow_datasets as tfds
from datetime import datetime
import pandas as pd
import numpy as np

### Checkup and linkage

In [6]:
torch.cuda.is_available()

True

## Datasets

### Billsum

In [7]:
billsumTest_url='https://drive.google.com/file/d/1Wd0M3qepNF6B4YwFYrpo7CaSERpudAG_/view?usp=share_link'
billsumTest_url='https://drive.google.com/uc?id=' + billsumTest_url.split('/')[-2]
billsum_test = pd.read_json(billsumTest_url, lines=True)
billsum_test = billsum_test.loc[:, ["text", "summary"]]
billsum_test.head(5)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Science Education Tax Incentive for B...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Small Business Expansion and Hiring Act of 201...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Requires the Director of National Intelligence...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,National Cancer Act of 2003 - Amends the Publi...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Military Call-up Relief Act - Amends the Inter...


### Gigaword

In [8]:
gigaword_builder = tfds.builder("Gigaword")
gigaword_builder.download_and_prepare()
gigaword = gigaword_builder.as_dataset()

In [9]:
gigaword = tfds.as_dataframe(gigaword["test"])
gigaword = gigaword.rename(columns={"document":"text", "summary":"summary"})
gigaword['summary'] = gigaword['summary'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword['text'] = gigaword['text'].str.decode("utf-8").str.strip().str.replace("#", "")
gigaword.head(5)

,text,summary
0,the german government and red cross have decid...,"germany gives , dollars in aid for iran quake ..."
1,teenager michaela krajicek claimed her first g...,krajicek defeats safina in dutch final UNK pic...
2,after spending years in prison for a killing ...,"$ , for years wrongly held in prison"
3,the first probable human case of mad cow disea...,st case of probable mad cow disease listed pos...
4,at least people were killed as typhoon sepat ...,"storms lash chinese coast killing ; , evacuat..."


### Multinews

In [10]:
multinews_builder = tfds.builder("MultiNews")
multinews_builder.download_and_prepare()
multinews = multinews_builder.as_dataset()

In [11]:
multinews = tfds.as_dataframe(multinews["test"])
multinews = multinews.rename(columns={"document":"text", "summary":"summary"})
multinews['summary'] = multinews['summary'].str.decode("utf-8").str.replace("\xe2\x80\x93", "").str[2:].str.strip()
multinews['text'] = multinews['text'].str.decode("utf-8").str.strip()
multinews.head(5)

,text,summary
0,"Think back, everyone -- can you remember when ...",No matter how much you like Jason Bateman and ...
1,'Alien: Covenant': Film Review \n \n Michael ...,A spaceship arrives on a distant planet that l...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,France is on lockdown today after a satirical ...
3,These crawls are part of an effort to archive ...,Just when you thought the Republican president...
4,The estranged wife of acclaimed concert pianis...,"Sofya Tsygankova, the estranged wife of famed ..."


### PubMed

In [36]:
pm_path = r"C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json"
pubmed_test = pd.read_json(pm_path, lines=True)
pm = pubmed_test[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()

pm.loc[:,"abstract_text"] = pm["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
pm.loc[:,"article_text"] = pm["article_text"].replace(regex=r"\[[^\]]*\]", value="")
pm.loc[:,"abstract_text"] = pm["abstract_text"].map(cleaner)
pm.loc[:,"article_text"] = pm["article_text"].map(cleaner)
pm = pm.rename(columns={"article_text":"text", "abstract_text":"summary"})
pubmed = pm.copy()

del pm 
del pubmed_test

pubmed.head()

,text,summary
0,anxiety affects quality of life in those livin...,research on the implications of anxiety in par...
1,small non - coding rnas are transcribed into m...,"small non - coding rnas include sirna , mirna ..."
2,ohss is a serious complication of ovulation in...,objective : to evaluate the efficacy and safet...
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


## Benchmark

In [13]:
results = {}
lim = 10

### Billsum

In [14]:
w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(billsum_test, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(billsum_test, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaM

In [15]:
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['billsum'] = billsum_results

In [16]:
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(billsum_test, limit=lim)

### Multinews

In [18]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(multinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(multinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(multinews, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaM

In [19]:
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['multinews'] = multinews_results

### Gigaword

In [20]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(gigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(gigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(gigaword, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaM

In [21]:
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['gigaword'] = gigaword_results

### PubMed

In [22]:
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(pubmed, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(pubmed, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(pubmed, limit=lim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaM

In [23]:
pubmed_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['pubmed'] = pubmed_results

## Evalutation

### Runtime Table

In [24]:
billsum_runtimes = [results["billsum"]["BERTScore"][1], results["billsum"]["Custom"][1], results["billsum"]["Bart"][1]]
multinews_runtimes = [results["multinews"]["BERTScore"][1], results["multinews"]["Custom"][1], results["multinews"]["Bart"][1]]
gigaword_runtimes = [results["gigaword"]["BERTScore"][1], results["gigaword"]["Custom"][1], results["gigaword"]["Bart"][1]]
pubmed_runtimes = [results["pubmed"]["BERTScore"][1], results["pubmed"]["Custom"][1], results["pubmed"]["Bart"][1]]

runtimeDf = pd.DataFrame({"Billsum": billsum_runtimes, 
                          "Multinews": multinews_runtimes,
                          "Gigaword": gigaword_runtimes,
                          "PubMed": pubmed_runtimes}, 
                         index=["Roberta-24-layers", "Word2Vec", "BART-large-CNN"])
runtimeDf

,Billsum,Multinews,Gigaword,PubMed
Roberta-24-layers,66.660384,37.605426,39.514590,38.881897
Word2Vec,27.907947,103.700297,0.080796,84.722921
BART-large-CNN,1.186100,1.280627,0.549545,1.418941


### Quality evaluation

#### Billsum

In [25]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"])),
                              axis=1)
billsumQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.542677,0.904048,0.678230,0.874369,0.704338,0.780197,0.392246
1,0.769379,0.953081,0.851434,0.846002,0.728316,0.782760,0.393373
2,0.718532,0.909344,0.802755,0.873844,0.702725,0.778998,0.338536
3,0.618058,0.895988,0.731513,0.810689,0.703450,0.753272,0.271333
4,0.480467,0.883006,0.622316,0.901381,0.665327,0.765571,0.481292


#### Multinews

In [26]:
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"])),
                              axis=1)
multinewsQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.648935,0.854094,0.737513,0.810646,0.794206,0.802342,0.259141
1,0.656565,0.880320,0.752154,0.841052,0.826047,0.833482,0.329445
2,0.651669,0.863386,0.742735,0.863537,0.827867,0.845326,0.340231
3,0.617941,0.856395,0.717885,0.862562,0.805295,0.832945,0.516795
4,0.687182,0.878509,0.771155,0.914194,0.850664,0.881285,0.547971


#### Gigaword

In [27]:
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"])),
                              axis=1)
gigawordQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.520848,0.858084,0.648229,0.931568,0.861043,0.894918,0.270875
1,0.426644,0.616523,0.504302,0.878367,0.817973,0.847095,0.373961
2,0.397880,0.683416,0.502947,0.882140,0.839132,0.860099,0.072829
3,0.622144,0.939688,0.748635,0.947691,0.876814,0.910876,0.444639
4,0.445092,0.737083,0.555027,0.878134,0.835666,0.856374,0.110215


#### PubMed

In [28]:
pubmedQualityDf = pd.concat((pd.DataFrame(results["pubmed"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["pubmed"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["pubmed"]["Bart"][0], columns=["Bart"])),
                              axis=1)
pubmedQualityDf.head(5)

,W2V_P,W2V_R,W2V_F,Bert_P,Bert_R,Bert_F,Bart
0,0.656127,0.952100,0.776879,0.854162,0.842896,0.848492,0.287621
1,0.601416,0.954034,0.737756,0.861836,0.812966,0.836688,0.385551
2,0.628735,0.966463,0.761847,0.806066,0.786019,0.795916,0.236114
3,0.662453,0.992789,0.794659,0.855671,0.825281,0.840201,0.439659
4,0.718312,0.963990,0.823212,0.828977,0.824318,0.826641,0.401576


#### Summary

In [29]:
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)
meanPubmedQualityDf = pubmedQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf,
                          "PubMed": meanPubmedQualityDf})
summaryDf

,Gigaword,Billsum,Multinews,PubMed
W2V_P,0.438547,0.630637,0.654471,0.660659
W2V_R,0.757547,0.907242,0.872364,0.965610
W2V_F,0.551814,0.741452,0.747495,0.780022
Bert_P,0.905045,0.852451,0.859192,0.829127
Bert_R,0.840408,0.696554,0.818171,0.807103
Bert_F,0.871343,0.765928,0.838067,0.817707
Bart,0.279085,0.374697,0.387969,0.309953


## Shuffled Data

### Shuffling datasets

In [40]:
shuffledGigaword = gigaword.copy()
shuffledGigaword["summary"] = shuffledGigaword["summary"].sample(frac=1).values

shuffledMultinews = multinews.copy()
shuffledMultinews["summary"] = shuffledMultinews["summary"].sample(frac=1).values

shuffledBillsum = billsum_test.copy()
shuffledBillsum["summary"] = shuffledBillsum["summary"].sample(frac=1).values

shuffledPubmed= pubmed.copy()
shuffledPubmed["summary"] = shuffledPubmed["summary"].sample(frac=1).values

In [41]:
shuffledMultinews.head(3)

,text,summary
0,"Think back, everyone -- can you remember when ...",President Trump ended his night by congratulat...
1,'Alien: Covenant': Film Review \n \n Michael ...,Those Brits who want to cut down on their alco...
2,By REUTERS \n \n Photo: Reuters \n \n Charli...,A New Jersey police officer has resigned after...


In [42]:
shuffledBillsum.head(3)

,text,summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Amends the Federal Law Enforcement Pay Reform ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Amends the Internal Revenue Code to allow an i...
2,SECTION 1. RELEASE OF DOCUMENTS CAPTURED IN IR...,Criminal Antitrust Anti-Retaliation Act - Proh...


In [43]:
shuffledGigaword.head(3)

,text,summary
0,the german government and red cross have decid...,fbi report shows mistreatment of guantanamo de...
1,teenager michaela krajicek claimed her first g...,dollar up gold falls UNK in graf that gold is...
2,after spending years in prison for a killing ...,saudi prince vows to fight terrorism after attack


In [44]:
shuffledPubmed.head(3)

,text,summary
0,anxiety affects quality of life in those livin...,this study was to determine the effects of fiv...
1,small non - coding rnas are transcribed into m...,peroxisome proliferator - activated receptor g...
2,ohss is a serious complication of ovulation in...,living in biofilms is probably the most common...


### Evaluating the forged datasets

In [46]:
results = {}
lim = 10

w2v = serialized_to_model(r'C:\Pro\Stages\A4 - DVRC\Work\Models\serialized_w2v.pkl')
bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledBillsum, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledBillsum, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledBillsum, limit=lim)
billsum_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['billsum'] = billsum_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledMultinews, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledMultinews, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledMultinews, limit=lim)
multinews_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['multinews'] = multinews_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledGigaword, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledGigaword, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledGigaword, limit=lim)
gigaword_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['gigaword'] = gigaword_results

bert_scores, bert_runtime = BERTScoreDynamicSampleTest(shuffledPubmed, limit=lim)
word2vec_scores, word2vec_runtime = BERTScoreStaticSampleTest(shuffledPubmed, w2v, lim, withIdf = False)
bart_scores, bart_runtime = BARTScoreDynamicSampleTest(shuffledPubmed, limit=lim)
pubmed_results = {'BERTScore': [bert_scores ,bert_runtime], 'Custom': [word2vec_scores, word2vec_runtime], 'Bart': [bart_scores, bart_runtime]}
results['pubmed'] = pubmed_results

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaM

In [47]:
billsumQualityDf = pd.concat((pd.DataFrame(results["billsum"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["billsum"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["billsum"]["Bart"][0], columns=["Bart"])),
                              axis=1)
multinewsQualityDf = pd.concat((pd.DataFrame(results["multinews"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["multinews"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["multinews"]["Bart"][0], columns=["Bart"])),
                              axis=1)
gigawordQualityDf = pd.concat((pd.DataFrame(results["gigaword"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["gigaword"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["gigaword"]["Bart"][0], columns=["Bart"])),
                              axis=1)
pubmedQualityDf = pd.concat((pd.DataFrame(results["pubmed"]["Custom"][0], columns=["W2V_P", "W2V_R", "W2V_F"]),
                              pd.DataFrame(results["pubmed"]["BERTScore"][0], columns=["Bert_P", "Bert_R", "Bert_F"]),
                              pd.DataFrame(results["pubmed"]["Bart"][0], columns=["Bart"])),
                              axis=1)
meanGigawordQualityDf = gigawordQualityDf.mean(axis=0)
meanMultinewsQualityDf = multinewsQualityDf.mean(axis=0)
meanBillsumQualityDf = billsumQualityDf.mean(axis=0)
meanPubmedQualityDf = pubmedQualityDf.mean(axis=0)

summaryDf = pd.DataFrame({"Gigaword":meanGigawordQualityDf,
                          "Billsum": meanBillsumQualityDf,
                          "Multinews": meanMultinewsQualityDf,
                          "Pubmed": meanPubmedQualityDf})
summaryDf

,Gigaword,Billsum,Multinews,Pubmed
W2V_P,0.250652,0.511985,0.520194,0.539729
W2V_R,0.335179,0.619819,0.668054,0.718335
W2V_F,0.283724,0.556002,0.583902,0.615668
Bert_P,0.808674,0.799225,0.800567,0.789745
Bert_R,0.794888,0.690427,0.769149,0.778015
Bert_F,0.801614,0.740694,0.784500,0.783664
Bart,0.108226,0.224518,0.206442,0.198792
